In [ ]:
import re

DigitSet = str  # e.g. '123'
Square   = str  # e.g. 'A9'
Picture  = str  # e.g. "53..7....6..195....98....6.8...6...34..8.3..17...2...6.6....28....419..5....8..79"
Grid     = dict # E.g. {'A9': '123', ...}, a dict  of {Square: DigitSet}
Fail     = Grid() # The empty Grid is used to indicate failure to find a solution

def cross(A, B) -> tuple:
    "Cross product of strings in A and strings in B."
    return tuple(a + b for a in A for b in B)

digits    = '123456789'
rows      = 'ABCDEFGHI'
cols      = digits
squares   = cross(rows, cols)
all_boxes = [cross(rs, cs)  for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
all_units = [cross(rows, c) for c in cols] + [cross(r, cols) for r in rows] + all_boxes
units     = {s: tuple(u for u in all_units if s in u) for s in squares}
peers     = {s: set().union(*units[s]) - {s} for s in squares}

def is_solution(solution: Grid, puzzle: Grid) -> bool:
    "Is this proposed solution to the puzzle actually valid?"
    return (solution is not Fail and
            all(solution[s] == puzzle[s] for s in squares if len(puzzle[s]) == 1) and
            all({solution[s] for s in unit} == set(digits) for unit in all_units))

In [ ]:
def parse(picture) -> Grid:
    """Convert a Picture to a Grid."""
    vals = re.findall(r"[.1-9]|[{][1-9]+[}]", picture)
    assert len(vals) == 81
    return {s: digits if v == '.' else re.sub(r"[{}]", '', v) 
            for s, v in zip(squares, vals)}

def picture(grid) -> Picture:
    """Convert a Grid to a Picture."""
    if grid is Fail: 
        return "Fail"
    def val(d: DigitSet) -> str: return '.' if d == digits else d if len(d) == 1 else '{' + d + '}'
    width = max(len(val(grid[s])) for s in grid)
    dash = '\n' + '+'.join(['-' * (width * 3 + 2)] * 3) + ' '
    def cell(r, c): return val(grid[r + c]).center(width) + ('|'  if c in '36' else ' ')
    def line(r): return ''.join(cell(r, c) for c in cols) + (dash if r in 'CF' else '')
    return '\n'.join(map(line, rows))

In [ ]:
grid1 = parse("53..7.... 6..195... .98....6. 8...6...3 4..8.3..1 7...2...6 .6....28. ...419..5 ....8..79")
#grid1 = parse("53..8.9.... 6..195... .98....6. 8...6...3 4..8.3..1 7...2...6 .6....28. ...419..5 ..8..79")
print(picture(grid1))

In [ ]:
def fill(grid, s, d) -> Grid:
    """Eliminate all the other digits (except d) from grid[s]."""
    if grid[s] == d or all(eliminate(grid, s, d2) for d2 in grid[s] if d2 != d):
        return grid
    else:
        return Fail

def eliminate(grid, s, d) -> Grid:
    """Eliminate d from grid[s]; implement the two constraint propagation strategies."""
    if d not in grid[s]:
        return grid        ## Already eliminated
    grid[s] = grid[s].replace(d, '')
    if not grid[s]:
        return Fail        ## Fail: no legal digit left
    elif len(grid[s]) == 1:
        # 1. If a square has only one possible digit, then eliminate that digit from the square's peers.
        d2 = grid[s]
        if not all(eliminate(grid, s2, d2) for s2 in peers[s]):
            return Fail    ## Fail: can't eliminate d2 from some square
    for u in units[s]:
        dplaces = [s for s in u if d in grid[s]]
        # 2. If a unit has only one possible square that can hold a digit, then fill the square with the digit.
        if not dplaces or (len(dplaces) == 1 and not fill(grid, dplaces[0], d)):
            return Fail    ## Fail: no place in u for d
    return grid

In [ ]:
def constrain(grid) -> Grid:
    "Propagate constraints on a copy of grid to yield a new constrained Grid."
    constrained: Grid = {s: digits for s in squares}
    for s in grid:
        d = grid[s]
        if len(d) == 1:
            fill(constrained, s, d)
    return constrained

In [ ]:
grid1 = parse("123123123 456456456 789789789 123123123 456456456 789789789 123123123 456456456 ....8..79")
print(picture(grid1))

In [ ]:
constrain(grid1)

In [ ]:
print(picture(constrain(grid1)))

In [ ]:
grid2 = parse("4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......")

print(picture(constrain(grid2)))

In [ ]:
def search(grid) -> Grid:
    "Depth-first search with constraint propagation (`fill`) to find a solution."
    if grid is Fail: 
        return Fail
    unfilled = [s for s in squares if len(grid[s]) > 1]
    if not unfilled: 
        return grid
    s = min(unfilled, key=lambda s: len(grid[s]))
    for d in grid[s]:
        solution = search(fill(grid.copy(), s, d))
        if solution:
            return solution
    return Fail

In [ ]:
def solve(puzzles, verbose=True) -> int:
    "Solve and verify each puzzle, and if `verbose`, print puzzle and solution."
    sep = '    '
    for puzzle in puzzles:
        solution = search(constrain(puzzle))
        assert is_solution(solution, puzzle)
        if verbose:
            print('\nPuzzle            ', sep, 'Solution')
            for p, s in zip(picture(puzzle).splitlines(), picture(solution).splitlines()):
                print(p, sep, s)
    return len(puzzles)

In [ ]:
solve([grid1, grid1])

In [ ]:
grid3 = parse("53..8.9.... 6..195... .98....6. 8...6...3 4..8.3..1 7...2...6 .6....28. ...419..5 ..8..79")


In [ ]:
solve([grid1, grid3])

In [ ]:
empty = parse('.' * 81)
              
solve([empty])